# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Маргасов Арсений Олегович`

Группа: `MADE-DS-11`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd
from itertools import count
from shapely.geometry import Point, Polygon
import random
import folium
import warnings
warnings.filterwarnings('ignore')

In [2]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [3]:
AfgPolygon = world.loc[world['name'] == 'Afghanistan']['geometry'][103]

In [4]:
AfgPolygon = Polygon(list(map(lambda x: (x[1], x[0]), AfgPolygon.exterior.coords[:])))

In [5]:
bbox = AfgPolygon.bounds
# afg_centroid = Point((bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2)
afg_centroid = AfgPolygon.centroid

In [6]:
random.seed = 19

In [7]:
def gen_point(prev_point):
    r = afg_centroid.distance(prev_point) / 4
    x0 = 0.25 * prev_point.x + 0.75 * afg_centroid.x
    y0 = 0.25 * prev_point.y + 0.75 * afg_centroid.y
    u = random.random()
    x = u * (prev_point.x + afg_centroid.x) / 2 + (1 - u) * afg_centroid.x
    y = y0 + np.sqrt(r * r - (x - x0) * (x - x0)) + np.random.normal(0, r)
    return Point(x, y)

In [9]:
points = []
p = Point(random.randint(-90, 90), random.randint(-180, 180))
for i in count(start = 1):
    print(p)
    points.append(p)
    if p.within(AfgPolygon):
        break
    p = gen_point(p)

POINT (-52 75)
POINT (13.20415084547464 125.313290120289)
POINT (27.63600424277515 78.83545258897658)
POINT (33.21362497696319 81.95198595888154)
POINT (33.65881081677772 72.78733559454433)
POINT (33.84386042496979 70.33894283145635)
POINT (33.85147904950278 69.6861971497687)


In [10]:
m = folium.Map(afg_centroid.coords[:][0], zoom_start=6)
folium.GeoJson(world.loc[world['name'] == 'Afghanistan']['geometry'][103]).add_to(m)
folium.Marker(
    location=points[0].coords[:][0],
    popup= 'Point #1' ,
    icon=folium.Icon(color='green', icon='info-sign')).add_to(m)
for i in range(1, len(points)):
    folium.Marker(
    location=points[i].coords[:][0],
    popup= 'Point #' + str(i + 1),
    icon=folium.Icon(color='green', icon='info-sign')).add_to(m)
    folium.PolyLine([points[i - 1].coords[:][0], 
                     points[i].coords[:][0]], color='purple').add_to(m)
folium.LatLngPopup().add_to(m)
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [173]:
import requests
import json
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
overpass = Overpass()

## Part 1

In [187]:
countries = overpass.query('relation["admin_level"="2"][boundary=administrative];out;', timeout=100)

In [226]:
countries_id_name = {}
for elem in countries.elements():
    if str(elem.id())[-1] == '0':
        countries_id_name[elem.id() + 3600000000] = elem.tag('name:en')

In [227]:
countries_id_realty_cnt = {}
for country_id in tqdm(countries_id_name.keys()):
    query = overpassQueryBuilder(area=country_id, elementType=['node'], selector='"building"="yes"', out='count')
    result = overpass.query(query, timeout=180)
    countries_id_realty_cnt[country_id] = result.countNodes()

100%|██████████| 27/27 [02:02<00:00,  4.55s/it]


In [232]:
import operator

In [238]:
max_realty_id = max(countries_id_realty_cnt.items(), key=operator.itemgetter(1))[0]

In [239]:
countries_id_name[max_realty_id]

'Australia'

In [240]:
query = overpassQueryBuilder(area=max_realty_id, elementType=['node'], selector='"amenity"="atm"')
result = overpass.query(query, timeout=180)

[overpass] downloading data: [timeout:180][out:json];area(3600080500)->.searchArea;(node["amenity"="atm"](area.searchArea);); out body;


In [266]:
afg_last = points[-1].coords[:][0]

In [300]:
from rtree import index
index = index.Index()

In [301]:
atm_id_coords = {}
for atm in result.elements():
    yx = tuple(reversed(atm.geometry()['coordinates']))
    atm_id_coords[atm.id()] = yx
    index.insert(atm.id(), yx + yx)

In [309]:
from geographiclib.geodesic import Geodesic

In [317]:
distances = []
for idx in index.nearest(afg_last + afg_last, 5):
    g = Geodesic.WGS84.Inverse(*(afg_last + atm_id_coords[idx]))
    distances.append(g['s12'])

In [319]:
# sum of distances to 5 nearest atms
sum(distances)

39980817.75001585

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [22]:
import pyproj
from shapely import geometry
from shapely.geometry import LineString, MultiPolygon
from openrouteservice import client
from OSMPythonTools.api import Api
from tqdm import tqdm

In [23]:
intersect_WaterWashingtonSt = Point(40.70320,-73.98958)
centralpark_avenue5th = Point(40.7649, -73.97256)

In [24]:
api_key = '5b3ce3597851110001cf6248f45bca95f4ec48a8839956dda7f75bd1'
clnt = client.Client(key=api_key)

In [107]:
request_params_1 = {'coordinates': [list(reversed(centralpark_avenue5th.coords[:][0])),
                                  list(reversed(intersect_WaterWashingtonSt.coords[:][0]))],
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false'}
route_normal = clnt.directions(**request_params_1)

In [123]:
route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.005)

In [38]:
loc_point = Point([(intersect_WaterWashingtonSt.coords[:][0][i] + centralpark_avenue5th.coords[:][0][i]) / 2 for i in range(2)])

In [47]:
schools = pd.read_csv('./2019_-_2020_School_Point_Locations.csv')
# schools['the_geom'] = schools['the_geom'].apply(lambda x: Point(float(x.split(' ')[2][:-1]), float(x.split(' ')[1][1:])))
schools['the_geom'] = schools['the_geom'].apply(lambda x: Point(float(x.split(' ')[1][1:]), float(x.split(' ')[2][:-1])))

In [109]:
def CreateBufferPolygon(point_in, resolution=2, radius=150):    

    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32632') # WGS84 UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution)

    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84

    return poly_wgs

In [110]:
# sites_poly = []
for _, school in tqdm(schools.iterrows()):
    site_coords = school['the_geom'].coords[:][0]
    site_poly_coords = CreateBufferPolygon(site_coords,
                                           resolution=2,
                                           radius=150)
#     sites_poly.append(site_poly_coords)

1992it [14:23,  2.31it/s]


In [124]:
sites_buffer_poly = []
sites_buffer_poly_idx = []
for idx, site_poly in enumerate(sites_poly):
    poly = Polygon(site_poly)
    if route_buffer.intersects(poly):
        sites_buffer_poly.append(poly)
        sites_buffer_poly_idx.append(idx)

In [125]:
request_params_2 = request_params_1.copy()
request_params_2['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(sites_buffer_poly))}
route_detour = clnt.directions(**request_params_2)

In [138]:
# GeoJSON style function
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=0.8)

map_params = {'tiles':'OpenStreetMap',
              'location':loc_point.coords[:][0],
              'zoom_start': 13}
map2 = folium.Map(**map_params)
for idx in sites_buffer_poly_idx:
    school = schools.iloc[idx, :]
    folium.features.Marker(list(reversed(school['the_geom'].coords[:][0])),
                           popup=school['Loc_Name'],
                           icon=folium.Icon(color='orange', icon='info-sign')).add_to(map2)
    
    site_poly_coords = [(y,x) for x,y in sites_poly[idx]]
    folium.vector_layers.Polygon(locations=site_poly_coords,
                                  color='purple',
                                  fill_color='purple',
                                  fill_opacity=1,
                                  weight=3).add_to(map2)

folium.features.GeoJson(data=route_normal,
                        name='Не жалеем школьников',
                        style_function=style_function('blue'),
                        overlay=True).add_to(map2)

folium.features.GeoJson(data=route_detour,
                        name='Объезжаем школы',
                        style_function=style_function('#00FF00'),
                        overlay=True).add_to(map2)

map2.add_child(folium.map.LayerControl())
map2